In [35]:
def hex_to_binary(hex_str,bit_length):
    """Convert hex string to binary string."""
    return str(bin(int(hex_str, 16))[2:].zfill(bit_length)) if bit_length else str(bin(int(hex_str, 16))[2:])

def binary_to_int(binary_str):
    return str(int(binary_str, 2))

def hex_to_decimal(hex_str):
    # 원래 16진수 문자열의 길이를 계산합니다.
    original_length = len(hex_str)
    # 16진수 문자열을 10진수 정수로 변환합니다.
    decimal_number = str(int(hex_str, 16))
    # 10진수 문자열의 길이를 계산합니다.
    decimal_length = len(decimal_number)
    # 원래 길이와 현재 길이를 비교하여 부족한 만큼 앞에 0을 추가합니다.
    if decimal_length < original_length:
        decimal_number = decimal_number.zfill(original_length)
    return decimal_number

def convert_lat(number_str):
    """
    8자리 숫자 문자열을 받아, ab를 정수 자리로, 
    cd.efgh를 소수로 변환하여 새로운 문자열로 된 숫자를 반환합니다.
    """
    if len(number_str) != 8 or not number_str.isdigit():
        raise ValueError("입력은 8자리 숫자 문자열이어야 합니다.")
    
    # ab를 정수 부분으로 추출
    integer_part = number_str[:2]
    
    # cd.efgh를 소수 부분으로 변환
    decimal_str = number_str[2:]
    decimal_part = int(decimal_str[:2]) + int(decimal_str[2:]) / 10000  # cd.efgh 형식으로 만듦
    decimal_part = decimal_part / 60  # 소수 부분을 60으로 나눔
    
    # 소수 부분을 문자열로 변환하고, "0."을 제거하여 결과 생성
    decimal_part_str = f"{decimal_part:.8f}".split('.')[1]
    
    # 최종 GPS 좌표 문자열 생성
    result = f"{integer_part}.{decimal_part_str}"
    
    return result

def convert_long(number_str):
    """
    9자리 숫자 문자열을 받아, abc를 정수 자리로,
    de.fghi를 소수로 변환하여 새로운 문자열로 된 숫자를 반환합니다.
    """
    if len(number_str) != 9 or not number_str.isdigit():
        raise ValueError("입력은 9자리 숫자 문자열이어야 합니다.")
    
    # abc를 정수 부분으로 추출
    integer_part = number_str[:3]
    
    # de.fghi를 소수 부분으로 변환
    decimal_de = int(number_str[3:5])
    decimal_fghi = int(number_str[5:]) / 10000  # fghi 부분을 소수로 변환
    decimal_part = (decimal_de + decimal_fghi) / 60  # 소수 부분을 60으로 나눔
    
    # 소수 부분을 문자열로 변환하고, "0."을 제거하여 결과 생성
    decimal_part_str = f"{decimal_part:.8f}".split('.')[1]
    
    # 최종 GPS 좌표 문자열 생성
    result = f"{integer_part}.{decimal_part_str}"
    
    return result

def parse_hexstring(hexstring):
    """ Parse the hexstring to extract the data fields. """
    data = {}
    data['DATA_LENGTH'] = len(hexstring)
    data['SOF'] = hexstring[0:2]
    # data['MSG_ID'] = hexstring[2:4]
    data['MSG_ID'] = 82
    data['Sequence'] = hexstring[6:8] + hexstring[4:6]
    data['PAYLOAD_LENGTH'] = hexstring[8:10]
    data['BUOY_ID'] = hexstring[24:26] + hexstring[22:24] + hexstring[20:22] + hexstring[18:20] + hexstring[16:18] + hexstring[14:16] + hexstring[12:14] + hexstring[10:12]
    data['BUOY_STATUS_HEX'] = hexstring[28:30] + hexstring[26:28]
    data['BUOY_STATUS_BINARY'] = hex_to_binary(hexstring[28:30],8)  + hex_to_binary(hexstring[26:28], 8)
    data['WIRELESS_STATUS'] = data['BUOY_STATUS_BINARY'][0:1]
    data['MAIN_MODULE_STATUS'] = data['BUOY_STATUS_BINARY'][1:2]
    data['PLORA_MODULE_STATUS'] = data['BUOY_STATUS_BINARY'][2:3]
    data['GNSS_STATUS'] = data['BUOY_STATUS_BINARY'][3:4]
    data['POWER'] = binary_to_int(data['BUOY_STATUS_BINARY'][4:7])
    data['LORA_STATUS'] = binary_to_int(data['BUOY_STATUS_BINARY'][7:10])
    data['PLORA_STATUS'] = binary_to_int(data['BUOY_STATUS_BINARY'][10:13])
    data['FISHINGBOAT_GW_STATUS'] = binary_to_int(data['BUOY_STATUS_BINARY'][13:16])
    data['YEAR'] = hex_to_decimal(hexstring[32:34] + hexstring[30])
    data['MONTH'] = hex_to_decimal(hexstring[31])
    data['DHM'] = hex_to_binary(hexstring[36:38],8) + hex_to_binary(hexstring[34:36],8)
    data['DAY'] = binary_to_int(data['DHM'][0:5])
    data['HOUR'] = int(binary_to_int(data['DHM'][5:10])) + int(binary_to_int('1001'))
    data['MIN'] = binary_to_int(data['DHM'][10:16])
    data['SECOND'] = binary_to_int(hex_to_binary(hexstring[38:40],0)[0:6])
    data['GPS_STATUS'] = hex_to_binary(hexstring[40:42],8)
    data['GNSS_STATUS'] = data['GPS_STATUS'][0]
    data['LATITUDE_CONDITION'] = data['GPS_STATUS'][1]
    data['LONGITUDE_CONDITION'] = data['GPS_STATUS'][2]
    data['GPS_MANUFACTURER'] = data['GPS_STATUS'][3:6]
    data['PRELIMINARY_VALUE'] = data['GPS_STATUS'][6:8]
    data['LATITUDE'] = convert_lat(str(int(hex_to_decimal(hexstring[48:50] + hexstring[46:48]))//2) + hex_to_decimal(hexstring[44:46] + hexstring[42:44]))
    data['LONGITUDE'] = convert_long(str(int(hex_to_decimal(hexstring[56:58] + hexstring[54:56]))//2) + hex_to_decimal(hexstring[52:54] + hexstring[50:52]))

    return data

In [36]:
# parse_hexstring('aa316808186dec20feff1d5788ad06697e486b80104300f01ab6248c62c7fe')
# parse_hexstring('aa316608186dec20feff1d5788ad06697e466b2c10d109f01a821b8c62c9fe')
# parse_hexstring('aa316708186dec20feff1d57886d06697e476b54108e04f01a2a208c6218fe')
a = parse_hexstring('aa31140018fded1dfeff63ca4000e6887ec7d2d810bd0b4a1c8f086463d1fe')
print(a["LATITUDE"])
print(a["LONGITUDE"])

36.35500833
127.37031833


In [10]:
from paho.mqtt.client import Client
import json
mqtt_client = Client()

MQTT_BROKER = '14.50.159.2'
MQTT_PORT = 1883
# MQTT_USER = 'rcnapp1@ttn'
# MQTT_PASSWORD = 'NNSXS.YCU7AAWPOCH27N6UEVVRZSF33IVIDMQRB6L2AHI.THLGQXBMXSESTGLXICJZQQGWJRMZAEARP7VQPTZI7SEYJHC7GLXA'

# mqtt_client.username_pw_set(MQTT_USER, MQTT_PASSWORD)
mqtt_client.connect(MQTT_BROKER, MQTT_PORT, 60)
MQTT_TOPIC = 'v3/rcnapp1@ttn/devices/105838541867477030805/up'
data = { # 위치 무궁화빌라
    "received_at": "2024-07-30 02:07:00", # test할 때 +9GMT 신경써서
    "frequency": "9552110",
    "hexString": 'aa31290018fded1dfeff63ca4000f8877ea4f224102413661cdc00826307fe',
    "dev_eui": "40ca63fffe1ded4a",
    "dev_addr": "11100009",
    "application_id":"appid123",
    "parsed_string": {
        "BUOY_ID": "40ca63fffe1ded4a",
        "BUOY_STATUS_BINARY": "1110100000000000",
        "BUOY_STATUS_HEX": "e800",
        "DATA_LENGTH": 62,
        "DAY": "10",
        "DHM": "0101001101110101",
        "FISHINGBOAT_GW_STATUS": "0",
        "GNSS_STATUS": "0",
        "GPS_MANUFACTURER": "100",
        "GPS_STATUS": "00010000",
        "HOUR": 22,
        "LATITUDE": "36.354856",
        "LATITUDE_CONDITION": "0",
        "LONGITUDE": "127.370261",
        "LONGITUDE_CONDITION": "0",
        "LORA_STATUS": "0",
        "MAIN_MODULE_STATUS": "1",
        "MIN": "53",
        "MONTH": "7",
        "MSG_ID": "31",
        "PAYLOAD_LENGTH": "18",
        "PLORA_MODULE_STATUS": "1",
        "PLORA_STATUS": "0",
        "POWER": "4",
        "PRELIMINARY_VALUE": "00",
        "SECOND": "40",
        "SOF": "aa",
        "Sequence": "0002",
        "WIRELESS_STATUS": "1",
        "YEAR": "2024"
    }
}
data2 = {
    "received_at": "2024-07-30 02:07:00", # test할 때 +9GMT 신경써서
    "frequency": "9552110",
    "hexString": 'aa313d0018fded1dfeff63ca4000f8877ec3f2cc102213661ca000826334fe',
    "dev_eui": "40ca63fffe1ded4b",
    "dev_addr": "11100009",
    "application_id":"appid123",
    "parsed_string": {
        "BUOY_ID": "40ca63fffe1ded4b",
        "BUOY_STATUS_BINARY": "1110100000000000",
        "BUOY_STATUS_HEX": "e800",
        "DATA_LENGTH": 62,
        "DAY": "10",
        "DHM": "0101001101110101",
        "FISHINGBOAT_GW_STATUS": "0",
        "GNSS_STATUS": "0",
        "GPS_MANUFACTURER": "100",
        "GPS_STATUS": "00010000",
        "HOUR": 22,
        "LATITUDE": "36.454164",
        "LATITUDE_CONDITION": "0",
        "LONGITUDE": "127.470301",
        "LONGITUDE_CONDITION": "0",
        "LORA_STATUS": "0",
        "MAIN_MODULE_STATUS": "1",
        "MIN": "53",
        "MONTH": "7",
        "MSG_ID": "31",
        "PAYLOAD_LENGTH": "18",
        "PLORA_MODULE_STATUS": "1",
        "PLORA_STATUS": "0",
        "POWER": "4",
        "PRELIMINARY_VALUE": "00",
        "SECOND": "40",
        "SOF": "aa",
        "Sequence": "0002",
        "WIRELESS_STATUS": "1",
        "YEAR": "2024"
    }
}
data3 = {
    "received_at": "2024-07-30 02:07:00", # test할 때 +9GMT 신경써서
    "frequency": "9552110",
    "hexString": 'aa313c0018fded1dfeff63ca4000f8877ec2f26810f812661c0501826319fe',
    "dev_eui": "40ca63fffe1ded4c",
    "dev_addr": "11100009",
    "application_id":"appid123",
    "parsed_string": {
        "BUOY_ID": "40ca63fffe1ded4c",
        "BUOY_STATUS_BINARY": "1110100000000000",
        "BUOY_STATUS_HEX": "e800",
        "DATA_LENGTH": 62,
        "DAY": "10",
        "DHM": "0101001101110101",
        "FISHINGBOAT_GW_STATUS": "0",
        "GNSS_STATUS": "0",
        "GPS_MANUFACTURER": "100",
        "GPS_STATUS": "00010000",
        "HOUR": 22,
        "LATITUDE": "36.254164",
        "LATITUDE_CONDITION": "0",
        "LONGITUDE": "127.270301",
        "LONGITUDE_CONDITION": "0",
        "LORA_STATUS": "0",
        "MAIN_MODULE_STATUS": "1",
        "MIN": "53",
        "MONTH": "7",
        "MSG_ID": "31",
        "PAYLOAD_LENGTH": "18",
        "PLORA_MODULE_STATUS": "1",
        "PLORA_STATUS": "0",
        "POWER": "4",
        "PRELIMINARY_VALUE": "00",
        "SECOND": "40",
        "SOF": "aa",
        "Sequence": "0002",
        "WIRELESS_STATUS": "1",
        "YEAR": "2024"
    }
}
mqtt_client.publish(MQTT_TOPIC, json.dumps(data))
mqtt_client.publish(MQTT_TOPIC, json.dumps(data2))
mqtt_client.publish(MQTT_TOPIC, json.dumps(data3))

In [106]:
import socket

# 대상 IP와 포트 설정
TARGET_IP = '115.71.11.205'  # 여기에 실제 IP 주소를 넣으세요.
TARGET_PORT = 9988           # 여기에 실제 포트 번호를 넣으세요.

def calculate_checksum(message_bytes):
    """
    메시지의 체크섬을 계산하는 함수.
    모든 바이트의 합을 1바이트로 반환.
    """
    checksum = sum(message_bytes) & 0xFF
    return checksum

def create_ship_message(sequence_number, msg_id, vessel_id, year, month, day, hour, minute, second, 
                        gps_status, latitude, latitude_frac, longitude, longitude_frac):
    """
    선박 정보를 포함한 메시지를 생성하는 함수.
    """
    # SOF (Start of Frame)
    sof = "aa"
    
    # Sequence Number (2바이트, 4자리 16진수)
    seq_num = f"{sequence_number:04x}"
    
    # Message ID (1바이트, 2자리 16진수)
    msg_id_hex = f"{msg_id:02x}"
    
    # Payload Length (고정 18 = 0x18)
    payload_length = "18"

    # Vessel ID (4바이트, 8자리 16진수)
    vessel_id_hex = f"{vessel_id:08x}"

    # Year/Month (2바이트, 4자리 16진수)
    year_month = (year << 4) | month
    year_month_hex = f"{year_month:04x}"

    # Day/Hour/Minute (2바이트, 4자리 16진수)
    day_hour_min = (day << 11) | (hour << 6) | minute
    day_hour_min_hex = f"{day_hour_min:04x}"

    # Second/Reserved (1바이트, 2자리 16진수)
    second_reserved = (second << 2)
    second_reserved_hex = f"{second_reserved:02x}".zfill(2)

    # GPS Status (1바이트, 2자리 16진수)
    gps_status_bin = (gps_status << 7) | (latitude_frac >> 2) | (longitude_frac >> 3)
    gps_status_hex = f"{gps_status_bin:02x}".zfill(2)

    # Latitude (위도 MSB + LSB, 4자리 16진수 + 4자리 16진수)
    latitude_msb = f"{latitude:04x}".zfill(4)
    latitude_lsb = f"{latitude_frac:04x}".zfill(4)[:4]  # 위도 소수 부분을 4자리로 자릅니다

    # Longitude (경도 MSB + LSB, 4자리 16진수 + 4자리 16진수)
    longitude_msb = f"{longitude:04x}".zfill(4)
    longitude_lsb = f"{longitude_frac:04x}".zfill(4)[:4]  # 경도 소수 부분을 4자리로 자릅니다

    # 메시지 구성
    message = (
        f"{sof}{msg_id_hex}{seq_num}{payload_length}{vessel_id_hex}"
        f"{year_month_hex}{day_hour_min_hex}{second_reserved_hex}"
        f"{gps_status_hex}{latitude_msb}{latitude_lsb}{longitude_msb}{longitude_lsb}"
    )

    # 필요한 길이를 계산하고 부족한 만큼 0으로 채우기
    desired_length = 58  # 전체 메시지의 길이가 58자리 (16진수)여야 합니다
    if len(message) < desired_length:
        padding = "0" * (desired_length - len(message))
        message = f"{sof}{msg_id_hex}{seq_num}{payload_length}{padding}{vessel_id_hex}" \
                  f"{year_month_hex}{day_hour_min_hex}{second_reserved_hex}" \
                  f"{gps_status_hex}{latitude_msb}{latitude_lsb}{longitude_msb}{longitude_lsb}"

    # 바이트 스트림으로 변환
    try:
        message_bytes = bytes.fromhex(message)
    except ValueError as ve:
        print(f"Error converting to bytes: {ve}")
        print(f"Message: {message}")
        raise

    # 체크섬 계산
    checksum = calculate_checksum(message_bytes)
    
    # 최종 메시지 생성
    final_message = f"{message}{checksum:02x}fe"
    
    return final_message

# 예제 데이터
sequence_number = 0x313d  # Sequence Number
msg_id = 0x43  # Message ID
vessel_id = 0xfded1dfe  # Vessel ID
year = 23  # 2023년
month = 8
day = 26
hour = 14
minute = 30
second = 45

# GPS 상태
gps_status = 0  # 정상
latitude = 0x3764  # 위도
latitude_frac = 0x63ca  # 위도 소수점 부분
longitude = 0x1273  # 경도
longitude_frac = 0x8400  # 경도 소수점 부분

# 메시지 생성
message = create_ship_message(sequence_number, msg_id, vessel_id, year, month, day, hour, minute, second, 
                                gps_status, latitude, latitude_frac, longitude, longitude_frac)

print(f"Generated Message: {message}")

# 데이터를 전송하는 부분
def send_data(data):
    try:
        # 소켓 생성
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            # 서버에 연결
            s.connect((TARGET_IP, TARGET_PORT))
            # 데이터를 바이너리로 인코딩 후 전송
            s.sendall(bytes.fromhex(data))
            print(f"Sent: {data}")
    except Exception as e:
        print(f"Failed to send data: {e}")


# 메시지 전송
send_data(message)


Generated Message: aa43313d180000000000fded1dfe0178d39eb418f2376463ca12738400f1fe
Sent: aa43313d180000000000fded1dfe0178d39eb418f2376463ca12738400f1fe


In [2]:
import datetime
import socket

# 데이터를 전송하는 부분
def send_data(data):
    try:
        TARGET_IP = '115.71.11.205'  # 여기에 실제 IP 주소를 넣으세요.
        TARGET_PORT = 9988           # 여기에 실제 포트 번호를 넣으세요.    
        # 소켓 생성
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            # 서버에 연결
            s.connect((TARGET_IP, TARGET_PORT))
            # 데이터를 바이너리로 인코딩 후 전송
            s.sendall(bytes.fromhex(data))
            print(f"Sent: {data}")
    except Exception as e:
        print(f"Failed to send data: {e}")

def calculate_checksum(hex_string):
    checksum = sum(bytearray.fromhex(hex_string)) % 256
    return f'{checksum:02x}'

def reverse_bytes(hex_string):
    # 바이트 단위로 역순으로 바꿔줌
    return ''.join([hex_string[i:i+2] for i in range(0, len(hex_string), 2)][::-1])

def print_binary(val, bits):
    return format(val, f'0{bits}b')

def encode_speed_course(speed, course):
    # 소수점 자리수는 고정 값으로 설정 (예: 소수점 이하 3자리)
    decimal_places_speed = 3
    decimal_places_course = 2

    # 속도 및 방향 정보를 28비트로 변환
    speed_value = int(speed * (10 ** decimal_places_speed))
    course_value = int(course * (10 ** decimal_places_course))

    # 속도 및 방향을 각각 32비트로 구성
    speed_hex = f'{(decimal_places_speed << 28) | speed_value:08x}'
    course_hex = f'{(decimal_places_course << 28) | course_value:08x}'

    # 바이트 순서 반전
    speed_hex = reverse_bytes(speed_hex)
    course_hex = reverse_bytes(course_hex)

    return speed_hex, course_hex

def encode_gps_data(sequence_num, ship_id, date_time, latitude_msb, latitude_lsb, longitude_msb, longitude_lsb, speed, course, gps_status, lat_dir, lon_dir, decimal_places):
    # 시퀀스 번호는 2바이트 (4자리 hex)
    seq_num_hex = f'{sequence_num:04x}'
    seq_num_hex = reverse_bytes(seq_num_hex)  # 엔디안 변환

    # 선박 ID는 4바이트 (8자리 hex)
    ship_id_hex = f'{ship_id:08x}'
    ship_id_hex = reverse_bytes(ship_id_hex)  # 엔디안 변환

    # 시간 정보 인코딩
    year = date_time.year  # 12 bits (0-99)
    month = date_time.month       # 4 bits (1-12)
    day = date_time.day           # 5 bits (1-31)
    hour = date_time.hour         # 5 bits (0-23)
    minute = date_time.minute     # 6 bits (0-59)
    second = date_time.second     # 6 bits (0-59)
    
    # 비트 연산을 통해 각 필드를 합침
    time_info_1 = (year << 4) | month
    time_info_2 = (day << 11) | (hour << 6) | minute
    time_info_3 = second << 2

    time_info_hex_1 = f'{time_info_1:04x}'
    time_info_hex_2 = f'{time_info_2:04x}'
    time_info_hex_3 = f'{time_info_3:02x}'

    time_info_hex_1 = reverse_bytes(time_info_hex_1)
    time_info_hex_2 = reverse_bytes(time_info_hex_2)

    # GPS 상태 정보 인코딩 (8바이트)
    gps_status_byte = (gps_status << 7) | (lat_dir << 6) | (lon_dir << 5) | (decimal_places << 2)
    gps_status_hex = f'{gps_status_byte:02x}'

    print(latitude_msb, latitude_lsb, longitude_msb, longitude_lsb)

    # 위도와 경도를 32비트로 구성 (15비트 정수, 17비트 소수)
    lat_hex = f'{latitude_msb:015b}{latitude_lsb:017b}'
    lon_hex = f'{longitude_msb:015b}{longitude_lsb:017b}'
    print(lat_hex, lon_hex)

    # 32비트 이진수를 8자리 16진수로 변환
    lat_hex = f'{int(lat_hex, 2):08x}'
    lon_hex = f'{int(lon_hex, 2):08x}'

    print(lat_hex, lon_hex)

    lat_hex = reverse_bytes(lat_hex)
    lon_hex = reverse_bytes(lon_hex)

    print(lat_hex, lon_hex)

    # 속도 및 방향 인코딩 (4바이트씩)
    speed_hex, course_hex = encode_speed_course(speed, course)

    # 페이로드 생성
    payload_hex = (
        f'{ship_id_hex}{time_info_hex_1}{time_info_hex_2}{time_info_hex_3}'
        f'{gps_status_hex}{lat_hex}{lon_hex}{speed_hex}{course_hex}'
    )

    # 페이로드 길이 계산
    payload_length = len(payload_hex) // 2
    payload_length_hex = f'{payload_length:02x}'

    # Checksum 계산
    full_payload_hex = f'aa43{seq_num_hex}{payload_length_hex}{payload_hex}'
    checksum_hex = calculate_checksum(full_payload_hex)

    # 최종 패킷 생성
    packet_hex = f'{full_payload_hex}{checksum_hex}fe'

    return packet_hex

# 예시 사용
sequence_num = 680
ship_id = 901
date_time = datetime.datetime.utcnow() + datetime.timedelta(hours=0)
latitude_msb = 3621
latitude_lsb = int(18041/6)
longitude_msb = 12722
longitude_lsb = int(12405/6)
speed = 0.23
course = 0
gps_status = 1
lat_dir = 0  # 북(N)
lon_dir = 0  # 동(E)
decimal_places = 4  # 소수점 자리수

packet_hex = encode_gps_data(sequence_num, ship_id, date_time, latitude_msb, latitude_lsb, longitude_msb, longitude_lsb, speed, course, gps_status, lat_dir, lon_dir, decimal_places)
print(packet_hex)
send_data(packet_hex)


3621 3006 12722 2067
00011100010010100000101110111110 01100011011001000000100000010011
1c4a0bbe 63640813
be0b4a1c 13086463
aa43a8021a85030000887e98e09c90be0b4a1c13086463e6000030000000202afe
Failed to send data: name 'TARGET_IP' is not defined


In [411]:

# 데이터를 전송하는 부분
def send_data(data):
    try:
        # 소켓 생성
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            # 서버에 연결
            s.connect((TARGET_IP, TARGET_PORT))
            # 데이터를 바이너리로 인코딩 후 전송
            s.sendall(bytes.fromhex(data))
            print(f"Sent: {data}")
    except Exception as e:
        print(f"Failed to send data: {e}")


# 메시지 전송
send_data('aa43a8021a85030000887e19dc5c90560b4a1c5f076463e6000030000000204afe')

Sent: aa43a8021a85030000887e19dc5c90560b4a1c5f076463e6000030000000204afe
